In [3]:
import numpy as np
import matplotlib.pyplot as plt
from pyuvdata import UVData

In [8]:
path = '/data4/paper/HERA19Golden/CalibratedData/2457548/LSThrs_20.0_23.0/'
uvd = UVData()
uvd.read_uvh5(path+'zen.2457548.55673_66112.xx.HH.uvcRPC.uvh5')

In [7]:
pwd

u'/data4/paper/HERA19Golden/CalibratedData/2457548'